In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the yelp business_attributes and business .csv files
business_attributes = pd.read_csv('yelp_business_attributes.csv')
yelp_business = pd.read_csv('yelp_business.csv')

In [3]:

def ll_columns(column_list):
    '''Function to display column names in a long list format'''
    colmns={}
    for idx, col in enumerate(column_list):
        colmns[idx] = col
    colmns=pd.DataFrame.from_dict(colmns, orient='index', columns=['column names'])
    colmns.index +=1                                
    return colmns
   

In [4]:
ll_columns(yelp_business.columns)

,column names
1,business_id
2,name
3,neighborhood
4,address
5,city
6,state
7,postal_code
8,latitude
9,longitude
10,stars


In [5]:
# Exploring the categories column
yelp_business.categories[0]

'Dentists;General Dentistry;Health & Medical;Oral Surgeons;Cosmetic Dentists;Orthodontists'

#### Functions for data filtering

The `nltk` library is used to create a function `find_words` that parses a string to find a specific word in this case restaurants or food. 

The function `review_biz` pulls data in chunks of 10000 rows and filters data based on their `business_ids`

In [6]:
# use nltk to determine which businesses are restaurants.
WNL = WordNetLemmatizer()

def find_words(text,word):
    '''This function takes in a string of words or 
    text and determines if word is found in the text '''
    
    word_tokens = word_tokenize(text)
    words = [WNL.lemmatize(w.lower()) for w in word_tokens if w.isalpha()]
    return word in words


# select all restaurants reviews in the dataset
def review_biz(filename):
    '''Function reads data from a file in chunks of 10000 lines and
    filters only data with ids found in restaurant_ids'''
    # Initialize reader object: user_reader
    review_reader = pd.read_csv(filename, chunksize=10000)
    
    # Initialize empty DataFrame: data
    review_data = pd.DataFrame()
   
    # Iterate over each DataFrame chunk
    for df_review in review_reader:
        restau_infos = df_review[df_review.business_id.isin(restaurant_ids)]
        review_data=review_data.append(restau_infos)
        
    return review_data

Use the `find_words` function and filter out business_ids that are associated with the word food or restaurant.

In [7]:
# Choose restaurants from the data set based on the words 'restaurant' or 'food'
# found in the 'categories' column

restaurant_ids = []

for idx, row in yelp_business.iterrows():
    if find_words(row.categories,'restaurant') or find_words(row.categories,'food'):
        restaurant_ids.append(row.business_id)


In [8]:
print(len(restaurant_ids))
restaurant_ids[:5]

69079


['PfOCPjBrlQAnz__NXj9h_w',
 'o9eMRCWt5PkpLDE0gOPtcQ',
 'EsMcGiZaQuG1OOvL9iUFug',
 'XOSRcvtaKc_Q5H1SAzN20A',
 'xcgFnd-MwkZeO5G2HQ0gAQ']

#### Filtering for business_ids in the US and IL
The list of all the states in the data set indicate state codes in different countries. A list of US state name codes is imported and used to filter only the states in the US and subsequently the state of Illinois (IL).

In [47]:
# list of the all the states in the dataset
yelp_business[yelp_business.business_id.isin(restaurant_ids)].state.unique()

array(['OH', 'BW', 'PA', 'ON', 'NC', 'SC', 'QC', 'AZ', 'NV', 'MLN', 'WI',
       'IL', 'EDH', 'CO', 'NYK', 'CHE', 'HLD', 'ABE', 'WLN', 'KHL', 'NY',
       'FIF', 'PKN', 'ST', '01', 'GLG', 'C', 'VS', 'ESX', 'ELN', 'IN',
       'CA', '30', 'NI', 'BY', 'AK', 'FLN', '6', 'B', 'VA', 'HU', '3',
       'XGL', 'ZET', 'RCC', 'WHT'], dtype=object)

In [9]:
# import list of US states and state name codes
us_states = pd.read_csv('us_states.txt', sep=' - ', header=None, engine='python') #engine=python because the 'sep= ' parameter would not take >1 character normally.
us_states.columns=['state_name','name_code']
us_states.head()

,state_name,name_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [11]:
# Create a new dataframe that holds all restaurant business_ids in the US.The state column has a state code found in 
# the list of us_states.

restaurants_us = yelp_business[yelp_business.business_id.isin(restaurant_ids) & yelp_business.state.isin(us_states.name_code)]

In [12]:
restaurants_us.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
8,EsMcGiZaQuG1OOvL9iUFug,"""Any Given Sundae""",NaN,"""2612 Brandt School Rd""",Wexford,PA,15090,40.615102,-80.091349,5.0,15,1,Coffee & Tea;Ice Cream & Frozen Yogurt;Food


In [13]:
# Total number of cities and list of US states found in the dataset.

print(restaurants_us.city.nunique())
restaurants_us.state.unique()


468


array(['OH', 'PA', 'NC', 'SC', 'AZ', 'NV', 'WI', 'IL', 'CO', 'NY', 'IN',
       'CA', 'AK', 'VA'], dtype=object)

#### Restaurants in IL:

`restaurants_us_IL` is created by filtering out only restaurants in Illinois

In [14]:
# Create dataframe of only restaurants in the Illinois.

restaurants_us_IL = restaurants_us[restaurants_us.state == 'IL']

print(restaurants_us_IL.business_id.nunique())
print(restaurants_us_IL.city.unique())

786
['Champaign' 'Urbana' 'Savoy' 'Mansfield' 'Tuscola' 'Saint Joseph'
 'Rantoul' 'Mahomet' 'Monticello' 'Tolono' 'Gifford' 'Philo' 'Ogden'
 'Homer' 'Oakwood' 'St Joseph' 'Camargo' 'Sidney' 'Fisher' 'Fithian'
 'Villa Grove' 'Mohamet']


Attributes of the restaurants are added to the `restaurants_us_IL` data frame by merging it with the `restaurants_attributes_IL` file which was filtered from the `business_attributes` data frame

In [15]:
# Get attributes of all restaurants in IL by merging the list 
# of restaurants to the business attributes

restau_attributes_IL = business_attributes[business_attributes.business_id.isin(restaurants_us_IL.business_id)]
restaurants_IL_atrbt = restaurants_us_IL.merge(restau_attributes_IL, on='business_id')

print(len(restau_attributes_IL))
print(len(restaurants_IL_atrbt))

763
763


In [16]:
restaurants_IL_atrbt.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,zV_aclADLjx2KOql9F_FTw,"""Crepe Cafe""",NaN,"""313 E Green St, Ste 5""",Champaign,IL,61820,40.109986,-88.233777,3.0,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,kyXEnWKQGWSThY6EcjORuw,"""Zorba's Restaurant""",NaN,"""627 E Green St""",Champaign,IL,61820,40.110085,-88.229304,4.0,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [17]:
# replace all Na with Nans
restaurants_IL = restaurants_IL_atrbt.replace('Na',np.nan)
restaurants_IL = restaurants_IL.dropna(axis=1, how='all')

In [18]:
 ll_columns(restaurants_IL.columns)

,column names
1,business_id
2,name
3,address
4,city
5,state
6,postal_code
7,latitude
8,longitude
9,stars
10,review_count


In [48]:
# Choosing relevant columns

restaurantz_IL = restaurants_IL[['business_id', 'name', 'city','state','postal_code','stars','review_count','categories']]
len(restaurantz_IL)

763

#### Importing informaiton from the large yelp_user file:
To import user information from the [yelp_user](https://www.yelp.com/dataset) file which is 1.6GB, the column headings are examined and a function created to filter in chunks of 10000 rows, just the user information relevant to the analysis.

In [49]:
# Check column headings in the yelp_user file which is 1.6GB
user_headings = pd.read_csv('yelp_user.csv', nrows=1).columns
ll_columns(user_headings)

,column names
1,user_id
2,name
3,review_count
4,yelping_since
5,friends
6,useful
7,funny
8,cool
9,fans
10,elite


In [50]:
# select information on all reviewers in the dataset

def users_stats(filename):
    '''Function reads data from a file in chunks if 10,000 lines and extracts the
    user_id, name, review_count and average_stars'''
    
    # Initialize reader object: user_reader
    user_reader = pd.read_csv(filename, chunksize=10000)
    
    # Initialize empty DataFrame: data
    user_info = pd.DataFrame()
   
    # Iterate over each DataFrame chunk
    for df_user in user_reader:
        user_metadata = df_user[['user_id','name','review_count','average_stars']]
        user_info=user_info.append(user_metadata)
        
    return user_info

In [22]:
%time user_info = users_stats('yelp_user.csv')

CPU times: user 1min 12s, sys: 6 s, total: 1min 18s
Wall time: 24.3 s


In [23]:
len(user_info)

1326100

In [24]:
user_info.head()

,user_id,name,review_count,average_stars
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,3.70
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2.00
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,4.67
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,4.67
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,3.45


#### Importing informaiton from the large yelp_review file:
Same as in the case of the `yelp_user` file, information on reviews per `business_id` is filtered from [yelp_review](https://www.yelp.com/dataset) file which is 3.79GB

In [51]:
# Determine column names in the yelp_review.csv file which is 3.79GB

reviews_headings = pd.read_csv('yelp_review.csv', nrows=1).columns
ll_columns(reviews_headings)

,column names
1,review_id
2,user_id
3,business_id
4,stars
5,date
6,text
7,useful
8,funny
9,cool


In [52]:
# select all restaurants review in the dataset
def review_biz(filename):
    '''pull data in chunks of 10000 lines and extract 5 columns for restaurants in IL only'''
    
    # Initialize reader object: user_reader
    review_reader = pd.read_csv(filename, chunksize=1000, usecols = ['review_id','user_id','business_id','stars','text'])
    
    # Initialize empty DataFrame: review_data
    review_data = pd.DataFrame()
   
    # Iterate over each DataFrame chunk
    for df_review in review_reader:
        restau_infos = df_review[df_review.business_id.isin(restaurantz_IL.business_id)]
        review_data=review_data.append(restau_infos)
    return review_data

In [27]:
# Get review information using review_biz function. yelp_review.csv is a 3.79GB file

%time restau_reviews_IL = review_biz('yelp_review.csv') #This takes about 2mins 28s to run

CPU times: user 2min 11s, sys: 13.8 s, total: 2min 25s
Wall time: 2min 26s


In [28]:
# save the restaurant or eateries reviews file for future use to minimize time.
restau_reviews_IL.to_csv('restau_reviews_IL.csv')

In [29]:
restau_reviews_IL.head(2)

,review_id,user_id,business_id,stars,text
1470,dfN6CDt6GVSOQjq6u8lYIw,4hnBlZWXN7fWoaP1HHNfgA,CpNMXASiwtJv5eCDf0n63g,4,"Solid steakhouse. Great atmosphere, can cook y..."
2095,aM9YAAnEy0g-htXRWzQtOg,PVyZXgOkVtnU6966FDFhuw,FTky74MxFIMvAJepeUUzEQ,3,The price is right and so is the location. Coo...


#### Merging Files
Check for missing values and rename columns to distinguish between restaurant ratings and user ratings before merging files.

In [30]:
# Determine if all the restaurant/eatery ratings are not having missing values.

restau_reviews_IL.stars.isnull().sum()

0

In [31]:
#Rename rating in restaurant review to user_restau_rating
restau_reviewz_IL = restau_reviews_IL.rename(index=str, columns={'stars':'user_restau_rating'})

restau_reviewz_IL.columns

Index(['review_id', 'user_id', 'business_id', 'user_restau_rating', 'text'], dtype='object')

In [32]:
#Rename review_count in user data to user_review_count 
user_data = user_info.rename(index=str, columns={'review_count':'user_review_count','name':'user_name','average_stars':'user_avg_stars'})
user_data.columns

Index(['user_id', 'user_name', 'user_review_count', 'user_avg_stars'], dtype='object')

In [33]:
#Rename review_count in restaurantz_IL to restau_review_count 
restaurantsz_IL = restaurantz_IL.rename(index=str, columns={'review_count':'restau_review_count', 'stars':'restau_rating', 'name':'restau_name'})
restaurantsz_IL.columns

Index(['business_id', 'restau_name', 'city', 'state', 'postal_code',
       'restau_rating', 'restau_review_count', 'categories'],
      dtype='object')

In [34]:
len(restaurantsz_IL)

763

In [35]:
#Merge Restaurant ratings restau_reviewz with user information from user_data.

restaurant_ratings_reviews_IL = restau_reviewz_IL.merge(user_data, on='user_id')

In [36]:
len(restaurant_ratings_reviews_IL)

26977

In [37]:
restaurants_data = restaurant_ratings_reviews_IL.merge(restaurantsz_IL, on='business_id')

In [38]:
restaurants_data.columns

Index(['review_id', 'user_id', 'business_id', 'user_restau_rating', 'text',
       'user_name', 'user_review_count', 'user_avg_stars', 'restau_name',
       'city', 'state', 'postal_code', 'restau_rating', 'restau_review_count',
       'categories'],
      dtype='object')

In [39]:
restaurants_data.describe()

,user_restau_rating,user_review_count,user_avg_stars,restau_rating,restau_review_count
count,26977.000000,26977.000000,26977.000000,26977.000000,26977.000000
mean,3.612781,102.966416,3.663734,3.634744,127.808207
std,1.368265,220.924721,0.741190,0.638081,152.964218
min,1.000000,1.000000,1.000000,1.000000,3.000000
25%,3.000000,8.000000,3.330000,3.500000,33.000000
50%,4.000000,25.000000,3.740000,3.500000,74.000000
75%,5.000000,93.000000,4.100000,4.000000,158.000000
max,5.000000,4338.000000,5.000000,5.000000,744.000000


In [40]:
restaurants_data.groupby(['city']).count().sort_values(by='user_restau_rating', ascending=False)
    

,review_id,user_id,business_id,user_restau_rating,text,user_name,user_review_count,user_avg_stars,restau_name,state,postal_code,restau_rating,restau_review_count,categories
city,,,,,,,,,,,,,,
Champaign,19682,19682,19682,19682,19682,19664,19682,19682,19682,19682,19645,19682,19682,19682
Urbana,5690,5690,5690,5690,5690,5689,5690,5690,5690,5690,5690,5690,5690,5690
Savoy,441,441,441,441,441,441,441,441,441,441,441,441,441,441
Mahomet,297,297,297,297,297,297,297,297,297,297,297,297,297,297
Rantoul,255,255,255,255,255,255,255,255,255,255,255,255,255,255
Tuscola,207,207,207,207,207,207,207,207,207,207,207,207,207,207
Monticello,189,189,189,189,189,189,189,189,189,189,189,189,189,189
Saint Joseph,65,65,65,65,65,65,65,65,65,65,65,65,65,65
St Joseph,33,33,33,33,33,33,33,33,33,33,33,33,33,33


`Mahomet` and `Mohamet` are very close in name and a search on Google indicates there is no city called `Mohamet` in IL. Checking the `postal_code` shows that they are identical.`Mohamet` is actually `Mahomet`.

In [41]:
# Determine if the postal code for Mahomet and Mohamet are the same. 
# If true then it is the same city. 

restaurants_IL[restaurants_IL.city=='Mahomet'].postal_code.unique() ==\
restaurants_IL[restaurants_IL.city=='Mohamet'].postal_code.unique()

array([ True])

In [42]:
# Replace Mohamet with Mahomet and count the number of reviews. It is 302 from 297
restaurants_data = restaurants_data.replace('Mohamet','Mahomet')
restaurants_data[restaurants_data.city=='Mahomet'].city.count()

302

In [43]:
restaurants_data.head(2)

,review_id,user_id,business_id,user_restau_rating,text,user_name,user_review_count,user_avg_stars,restau_name,city,state,postal_code,restau_rating,restau_review_count,categories
0,dfN6CDt6GVSOQjq6u8lYIw,4hnBlZWXN7fWoaP1HHNfgA,CpNMXASiwtJv5eCDf0n63g,4,"Solid steakhouse. Great atmosphere, can cook y...",Sheryl,3,4.67,"""Alexander's Steakhouse""",Champaign,IL,61821,3.0,62,Steakhouses;Restaurants;Seafood
1,9bkef_jblUrbIUZvgdzJuQ,iLJbk0VJ2gdF4NfHPNWNHQ,CpNMXASiwtJv5eCDf0n63g,2,Too pricey when you have to cook your own meat...,Michael,27,3.55,"""Alexander's Steakhouse""",Champaign,IL,61821,3.0,62,Steakhouses;Restaurants;Seafood


#### The pre-wrangled data is saved as `restaurants_all_info_IL.csv` 


In [44]:
restaurants_data.to_csv('restaurants_all_info_IL.csv')

Further analyses is done on the saved file in the [ylp_recommends_us_II](ylp_recommends_us_II_pearson.ipynb) jupyter notebook